# About this notebook

@author: Yingding Wang\
@created: 19.Dec 2023\
@upated: 19.Dec 2023\
@version: 1

This notebook demonstrate the use of postgres client to connect to a postgres db.

* postgres client (psycopg): https://www.psycopg.org/
* Documentation (psycopg): https://www.psycopg.org/docs/usage.html

In [1]:
import sys, os
# !{sys.executable} -m pip install --upgrade --use-feature=2020-resolver psycopg2-binary 2.9.9 python-dotenv==1.0.0
!{sys.executable} -m pip install --user --upgrade psycopg2-binary==2.9.9 python-dotenv==1.0.0

In [2]:
import psycopg2
from typing import Dict, List, Tuple
from psycopg2.extensions import cursor

## (Option 1) Edit postgres.env file for DB connection

In [3]:
# ENV_FILE="postgres.env"

### Uncomment the following cell and edit with your PostresDB credential to connect to DB

In [4]:
'''
%%writefile $ENV_FILE
# environment variables for Postgres DB 14.1 
DB_HOST="Database_Host_DNS_NAME"
DB_NAME="Database_Name"
DB_PORT="Database_Port"
DB_USER="Database_Login_User_Name"
DB_USER_PW="Database_Login_Password"
'''

'\n%%writefile $ENV_FILE\n# environment variables for Postgres DB 14.1 \nDB_HOST="Database_Host_DNS_NAME"\nDB_NAME="Database_Name"\nDB_PORT="Database_Port"\nDB_USER="Database_Login_User_Name"\nDB_USER_PW="Database_Login_Password"\n'

In [5]:
# from dotenv import load_dotenv
# load_dotenv(dotenv_path="postgres.env", override=True)

"""
print(f"\
{os.environ['DB_HOST']}\n\
{os.environ['DB_NAME']}\n\
{os.environ['DB_PORT']}\n\
{os.environ['DB_USER']}\n\
{os.environ['DB_USER_PW']}\n\
")
"""

'\nprint(f"{os.environ[\'DB_HOST\']}\n{os.environ[\'DB_NAME\']}\n{os.environ[\'DB_PORT\']}\n{os.environ[\'DB_USER\']}\n{os.environ[\'DB_USER_PW\']}\n")\n'

In [6]:
# db_conf_dict = {
#     "dbname":  os.environ['DB_NAME'],
#     "user":    os.environ['DB_USER'],
#     "password":os.environ['DB_USER_PW'],
#     "host":    os.environ['DB_HOST'],
#     "port": os.environ['DB_PORT']  
# }
# print(db_conf_dict)

## (Option 2) Loading credentials from ENV variables
The env variables can be injected from Kubeflow PodDefault

In [7]:
'''Uncomment this cell if you are using opton1'''
from dataclasses import dataclass
@dataclass
class DBConfig:   
    # DB_HOST: str = ""
    # DB_USER: str = ""
    # DB_USER_PW: str = ""
    # DB_NAME: str = ""
    # DB_SCHEMA: str = ""
    # DB_STAGING_PORT: int = 5432
    # DB_ANALYTICS_PORT: int = 5432
    

    def __post_init__(self):
        self.DB_HOST = self._load_env("SCIVIAS_DB_HOST")
        self.DB_USER = self._load_env("SCIVIAS_DB_USERNAME")
        self.DB_USER_PW = self._load_env("SCIVIAS_DB_PASSWORD")
        self.DB_NAME = self._load_env("SCIVIAS_DB_NAME")
        self.DB_SCHEMA = self._load_env("SCIVIAS_DB_SCHEMA")
        self.DB_STAGING_PORT = self._load_env("SCIVIAS_STAGING_DB_PORT", 5432)
        self.DB_ANALYTICS_PORT = self._load_env("SCIVIAS_ANALYTICS_DB_PORT", 5432)
    
    
    @classmethod
    def _load_env(clz, key: str, default = ""):
        return os.environ.get(key, default)
    
    
    def __repr__(self) -> str:
        """for program and developer obj representation also used by jupyter notebook cell"""
        return f"{self.DB_HOST}\n{self.DB_USER}\n{self.DB_USER_PW}\n{self.DB_NAME}\n{self.DB_SCHEMA}\n{self.DB_STAGING_PORT}\n{self.DB_ANALYTICS_PORT}"
        
    def __str__(self):
        """for user print() function"""
        return self.__repr__()
    
    
    def to_psycopg_dic(self, is_analytics: True) -> dict:
        return {
            "dbname":  self.DB_NAME,
            "user":    self.DB_USER,
            "password": self.DB_USER_PW,
            "host":     self.DB_HOST,
            "port":  self.DB_ANALYTICS_PORT if is_analytics else self.DB_STAGING_PORT,   
        }

    
db_conf_obj = DBConfig()
db_conf_dict = db_conf_obj.to_psycopg_dic(is_analytics=True)

In [8]:
def run_sql(config: Dict, sql_statement: str) -> None:
    """run sql without any return """
    with psycopg2.connect(**config) as conn:
        with conn.cursor() as curs:
            try:
                curs.execute(sql_statement)
            except Exception as cause:
                print(f"{cause}, {type(cause)}")

                
# use a function decorator
def run_sql_cursor(func):
    def inner(config:Dict, sql_statement: str):
        with psycopg2.connect(**config) as conn:
            with conn.cursor() as curs:
                try:
                    # print(type(curs))
                    return func(config, sql_statement, curs)
                except Exception as cause:
                    print(f"{cause}, {type(cause)}")
    return inner


@run_sql_cursor
def run_sql_with_return(config: Dict, sql_statement: str, curs: cursor=None) -> list:
    """run sql with result returned as a list of tuple"""
    if (curs is not None):
        curs.execute(sql_statement)
        # curs.fetchall() get a list of tuple https://www.psycopg.org/docs/cursor.html#cursor.fetchall
        result: List[Tuple] = curs.fetchall()
        # print list pretty: https://www.geeksforgeeks.org/print-lists-in-python-4-different-ways/    
        # for el in result:
        #    print(*el, sep=",") # print tuple
    else:
        result: List = [()]
    return result


def print_tuple_list(list: List[Tuple]) -> None:
    for el in list:
        print(*el, sep=",") # print tuple

In [9]:
# create table with name test if not exit
# sql1="CREATE TABLE IF NOT EXISTS test (id serial PRIMARY KEY, num integer, data varchar);"

In [ ]:
# run_sql(config, "delete from llm_med_report_info where run_name = '';")

In [10]:
# run_sql(config, sql1)

## Run SQL with results

In [11]:
# https://stackoverflow.com/a/24008869
sql_show_tables="select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';"

In [12]:
result = run_sql_with_return(db_conf_dict, sql_show_tables)
print_tuple_list(result)

lab_data
test_user_report_info


In [13]:
result = run_sql_with_return(db_conf_dict, "select count(*) from lab_data")
print_tuple_list(result)

236884


In [18]:
stmt_str = "select count(*) from test_user_report_info"
# stmt_str = "select * from test_user_report_info"
result = run_sql_with_return(db_conf_dict, stmt_str)
print_tuple_list(result)

2
